In [2]:
import numpy as np
import pandas as pd
from scipy.signal import correlate
import matplotlib.pyplot as plt
from scipy.sparse import lil_matrix
from itertools import product
from itertools import combinations
from datetime import datetime, timedelta
from multiprocessing import Pool


# Load the temporal series data
data_with_date = pd.read_csv('temporal_series_21.csv')
data = data_with_date.drop(columns=['date'])

#Inizialize parameters
max_tau = 15 
threshold = 0.05

#Function to compute maximum of time correlation function between two temporal series
def time_delayed_cross_correlation(df, col1, col2, max_tau, threshold=0.05):
    """
    Optimized custom time-delayed cross-correlation function for two time series.

    Args:
    df (pandas.DataFrame): Dataframe containing the time series data.
    col1 (str): Name of the first time series column.
    col2 (str): Name of the second time series column.
    max_tau (int): Maximum time delay (positive or negative).
    threshold (float): Threshold for maximum correlation.

    Returns:
    float: Maximum cross-correlation above the threshold, or 0 if none.
    """
    # Extract the series and compute means and std deviations
    series_i = df[col1].values
    series_j = df[col2].values
    mean_i, std_i = series_i.mean(), series_i.std()
    mean_j, std_j = series_j.mean(), series_j.std()
    
    # Check if standard deviations are zero to avoid division by zero
    if std_i == 0 or std_j == 0:
        return 0
    
    # Initialize array to store cross-correlations for each lag
    cross_corrs = np.zeros(2 * max_tau + 1)

    # Compute cross-correlation for each lag
    for tau in range(-max_tau, max_tau + 1):
        if tau < 0:
            # Positive lag: shift series_j forward by -tau (series_i aligns with delayed series_j)
            numerator = np.sum((series_i[-tau:] - mean_i) * (series_j[:len(series_j) + tau] - mean_j))
            denominator = (std_i * std_j * (len(series_i) + tau))
        else:
            # Negative or zero lag: shift series_i forward by tau (series_j aligns with delayed series_i)
            numerator = np.sum((series_i[:len(series_i) - tau] - mean_i) * (series_j[tau:] - mean_j))
            denominator = (std_i * std_j * (len(series_i) - tau))

        # Calculate cross-correlation, check for zero denominator
        cross_corrs[tau + max_tau] = numerator / denominator if denominator != 0 else 0

    # Find the maximum correlation and compare with threshold
    max_corr = np.max(cross_corrs)
    max_index = np.argmax(cross_corrs)
    best_tau = max_index - max_tau  # Convert array index back to tau value

    # Apply threshold check and adjust max_corr based on tau direction
    if max_corr >= threshold:
        return max_corr if best_tau >= 0 else -max_corr
    else:
        return 0


#Function to compute the whole adjacency matrix (impossible to compute)
def ccr_matrix(df):
    #build the adjacency matrix
    adjacency_matrix = pd.DataFrame(np.nan, index=df.columns, columns=df.columns)

    #iterate for each pair of columns
    for col1 in df.columns:
        for col2 in df.columns:
            adjacency_matrix.loc[col1,col2] = time_delayed_cross_correlation(df, col1, col2, max_tau)

    return adjacency_matrix

#function to compute just a submatrix of the adjacency matrix (submatricese on the diagonal)
def compute_submatrix(df, columns, filename, max_tau):
    submatrix = pd.DataFrame(index=columns, columns=columns)

    # Loop through column pairs, skipping redundant calculations
    for col1, col2 in product(columns, repeat=2):
        if col1 <= col2:  # Ensures each pair is calculated only once
            correlation = time_delayed_cross_correlation(df, col1, col2, max_tau)
            submatrix.loc[col1, col2] = correlation
            submatrix.loc[col2, col1] = correlation  # Fill symmetric position

    submatrix.to_csv(filename)
    print(f"Submatrix {filename} has been saved.")
    return submatrix

#function to computer a submatrix which is not on the diagonal
def compute_cross_group_matrix(df, group_a, group_b, filename):
    cross_matrix = pd.DataFrame(index=group_a, columns=group_b)
    for col1, col2 in product(group_a, group_b):
        cross_matrix.loc[col1, col2] = time_delayed_cross_correlation(df, col1, col2, max_tau)
    cross_matrix.to_csv(filename)
    print(f"submatrix {filename} has been saved")
    return cross_matrix

#Now to compute the submatrices we need to divide the 3104 columns into groups
def divide_into_groups(df, group_sizes):
    #Divides the comlumns into specified sizes
    columns = df.columns.tolist()
    groups = []
    start = 0
    for size in group_sizes:
        groups.append(columns[start:start + size])
        start += size
    return groups

#Change of program, now we will try to compute manually the single pieces
group_1 = data.columns[0:1000]
group_2 = data.columns[1000:2000]
group_3 = data.columns[2000:3104]


filename_sub_1 = "21_sub_mat_1.csv"
compute_submatrix(data, group_1, filename_sub_1, max_tau)

filename_sub_2 = "21_sub_mat_2.csv"
compute_submatrix(data, group_2, filename_sub_2, max_tau)

filename_sub_3 = "21_sub_mat_3.csv"
compute_submatrix(data, group_3, filename_sub_3, max_tau)

filename_cross_1 = "21_cross_matrix_1_2.csv"
compute_cross_group_matrix(data, group_1, group_2, filename_cross_1)

filename_cross_2 = "21_cross_matrix_1_3.csv"
compute_cross_group_matrix(data, group_1, group_3, filename_cross_2)

filename_cross_3 = "21_cross_matrix_2_3.csv"
compute_cross_group_matrix(data, group_2, group_3, filename_cross_3)


Submatrix 20_sub_mat_1.csv has been saved.
Submatrix 20_sub_mat_2.csv has been saved.
Submatrix 20_sub_mat_3.csv has been saved.
submatrix 20_cross_matrix_1_2.csv has been saved
submatrix 20_cross_matrix_1_3.csv has been saved
submatrix 20_cross_matrix_2_3.csv has been saved


38099.0   38101.0   38103.0   38105.0   39001.0   39003.0   39005.0  \
21083.0 -0.655377 -0.708758  -0.45659  -0.74754 -0.662124 -0.619365 -0.576808   
21085.0 -0.728168 -0.789685 -0.464874 -0.840437 -0.696548 -0.606745 -0.618567   
21087.0 -0.700843 -0.772981 -0.443319 -0.763209 -0.603544 -0.514825 -0.476874   
21089.0  0.607883  0.707296 -0.484008  0.751027  0.635214 -0.549551 -0.565283   
21091.0  0.649926  0.697767 -0.469535  0.718109 -0.665949 -0.589397  -0.61114   
...           ...       ...       ...       ...       ...       ...       ...   
38089.0  0.764184  0.819804 -0.556154  0.830427 -0.642675 -0.597954 -0.600224   
38091.0  0.580926 -0.512627 -0.504213  0.578709 -0.530917 -0.500658 -0.436456   
38093.0  0.831879  0.856938 -0.514537 -0.882909 -0.710925  -0.72146 -0.621222   
38095.0  0.546436 -0.587656 -0.430586  0.631372 -0.513515  0.449155 -0.423955   
38097.0  0.781218  0.750772 -0.522765  0.770572  0.478101 -0.563818    -0.436   

          39007.0   39009.0   39011.0  ...   56027.0   56029.0   56031.0  \
21083.0 -0.565858 -0.652304 -0.616041  ...  0.430022  0.685545  0.634655   
21085.0 -0.564862 -0.666791 -0.704248  ...  0.467627  0.747635  0.676704   
21087.0 -0.509514  -0.59554 -0.650667  ...  0.447894  0.621858  0.549996   
21089.0 -0.542738 -0.618551 -0.586965  ...  0.519362  0.700093  0.654544   
21091.0 -0.549421 -0.619095 -0.647682  ...  0.400229  0.678678   0.58592   
...           ...       ...       ...  ...       ...       ...       ...   
38089.0 -0.563505 -0.651449 -0.737798  ...  0.432048 -0.721032 -0.685558   
38091.0 -0.361433 -0.583363 -0.534959  ...  0.383733 -0.440531 -0.440199   
38093.0 -0.658627  -0.77818 -0.837164  ...   0.44986 -0.589805 -0.494667   
38095.0 -0.470726  0.477784 -0.515028  ...  0.288746 -0.539038  -0.42673   
38097.0 -0.401344 -0.615757 -0.633316  ...  0.469195 -0.593384  0.614358   

          56033.0   56035.0   56037.0   56039.0   56041.0   56043.0   56045.0  
21083.0  0.864187  0.724601  0.843793    0.6868  0.883736  0.508174  0.547443  
21085.0  0.908916  0.804977  0.889048 -0.706623  0.895163  0.647788  0.671475  
21087.0  0.793717  0.713379  0.777501    -0.613  0.811513  0.554381  0.568665  
21089.0  0.850527   0.76336  0.783343  0.785284  0.856078  0.535917  0.568366  
21091.0  0.803982  0.702644  0.807437  0.707933  0.851191  0.522344  0.541598  
...           ...       ...       ...       ...       ...       ...       ...  
38089.0 -0.825685 -0.781951 -0.764408  -0.81248 -0.776016   0.72991  0.689578  
38091.0 -0.564783 -0.500172 -0.548327 -0.549543 -0.526472  0.431038  0.370704  
38093.0  -0.73459 -0.754042 -0.732134 -0.762916 -0.806974  0.611636  0.604687  
38095.0 -0.583859   0.62375 -0.569542 -0.535665  0.592856   0.44738  0.526995  
38097.0  0.654088  0.738313  0.718955 -0.561956 -0.680303  0.603981  0.588564  

[1000 rows x 1104 columns]